# Advent of Code: Day 8

## Part One

--- Day 8: Haunted Wasteland ---

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

```
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
```

Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

```
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
```

Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?



In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from functools import cached_property
from itertools import cycle, chain, repeat
from math import gcd
import re

In [ ]:
@dataclass
class Connection:
    node: str
    left: str
    right: str


@dataclass
class Document:
    instructions: list[str]
    network: dict[str, Connection]

    def get_key(self, key: str, instruction: str) -> str:
        connection: Connection = self.network[key]
        if instruction == 'L':
            return connection.left
        elif instruction == 'R':
            return connection.right
        raise ValueError(f'Unhandled instruction: "{instruction}"')

    def get_keys(self) -> list[str]:
        return list(self.network.keys())


    @staticmethod
    def create_from_lines(lines: list[str]) -> Document:
        instructions = [ch for ch in lines[0]]
        pattern: str = r'''
            (?P<node>\w{3})
            \s=\s
            \(
            (?P<left>\w{3})
            ,\s
            (?P<right>\w{3})
            \)
        '''
        network: dict[str, Connection] = {}
        p = re.compile(pattern, re.MULTILINE | re.VERBOSE)
        for line in lines[1:]:
            line = line.strip()
            if len(line) > 0:
                match = p.match(line)
                if match is not None:
                    # print(match)
                    network[match.group('node')] = Connection(node=match.group('node'), left=match.group('left'), right=match.group('right'))
        return Document(instructions=instructions, network=network)


def parse_input(lines: list[str]) -> Document:
    return Document.create_from_lines()


def get_input_small() -> list[str]:
    return """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)""".split('\n')


def get_input_medium() -> list[str]:
    return """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)""".split('\n')


def get_input() -> list[str]:
    with open('20231208.txt', 'r') as f:
        return [line.strip() for line in f.readlines()]


def solution_part1():
    lines: list[str]
    # lines = get_input_small()
    # lines = get_input_medium()
    lines = get_input()
    # lines: list[str] = get_input()
    doc: Document = Document.create_from_lines(lines)
    current_key: str = 'AAA'
    for idx, instruction in enumerate(cycle(doc.instructions), start=1):
        next_key = doc.get_key(current_key, instruction)
        # print(idx, instruction, current_key, next_key)
        if next_key == 'ZZZ':
            return idx
        current_key = next_key
        
    return 0


solution_part1()

--- Part Two ---

The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:

```
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
```

Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

- Step 0: You are at 11A and 22A.
- Step 1: You choose all of the left paths, leading you to 11B and 22B.
- Step 2: You choose all of the right paths, leading you to 11Z and 22C.
- Step 3: You choose all of the left paths, leading you to 11B and 22Z.
- Step 4: You choose all of the right paths, leading you to 11Z and 22B.
- Step 5: You choose all of the left paths, leading you to 11B and 22C.
- Step 6: You choose all of the right paths, leading you to 11Z and 22Z.

So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [ ]:
@dataclass
class Document:
    instructions: list[str]
    network: dict[tuple[str, str], str]
    start_keys: list[str]
    end_keys: list[str]

    def get_key(self, key: str, instruction: str) -> str:
        return self.network[(key, instruction)]

    def get_keys(self) -> set[str]:
        return set([k for k, instruction in self.network.keys()])

    @staticmethod
    def create_from_lines(lines: list[str]) -> Document:
        instructions = [ch for ch in lines[0]]
        pattern: str = r'''
            (?P<node>\w{3})
            \s=\s
            \(
            (?P<left>\w{3})
            ,\s
            (?P<right>\w{3})
            \)
        '''
        network: dict[tuple[str, str], str] = {}
        p = re.compile(pattern, re.MULTILINE | re.VERBOSE)
        for line in lines[1:]:
            line = line.strip()
            if len(line) > 0:
                match = p.match(line)
                if match is not None:
                    # print(match)
                    node = match.group('node')
                    left = match.group('left')
                    right = match.group('right')
                    network[(node, 'L')] = left
                    network[(node, 'R')] = right
        start_keys: list[int] = set([k for k, instruction in network.keys() if k.endswith('A')])
        end_keys: list[int] = set([k for k, instruction in network.keys() if k.endswith('Z')])
        return Document(instructions=instructions, network=network, start_keys=start_keys, end_keys=end_keys)


def parse_input(lines: list[str]) -> Document:
    return Document.create_from_lines()

def get_input_part2():
    return """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)""".split('\n')


def solution_part2_slow():
    lines: list[str]
    # lines = get_input_part2()
    # lines = get_input_small()
    # lines = get_input_medium()
    lines = get_input()
    doc: Document = Document.create_from_lines(lines)
    print(f'{len(doc.instructions)} instructions')
    print(f'start_keys: {doc.start_keys}')
    print(f'end_keys: {doc.end_keys}')
    current_keys: list[int] = doc.start_keys
    next_keys: list[str]
    n_print_every: int = 100000 * len(doc.instructions)
    n_limit: int = n_print_every * 100
    for idx, instruction in enumerate(cycle(doc.instructions), start=1):
        next_keys = [doc.get_key(current_key, instruction) for current_key in current_keys]
        # if idx % n_print_every == 0:
        #     print(idx, instruction, current_keys, next_keys)
        matching_keys = [k for k in next_keys if k in doc.end_keys]
        if len(matching_keys) > 2:
            print(f'{idx}: {len(matching_keys)} end keys: {matching_keys}')
        if set(next_keys).issubset(set(doc.end_keys)):
            return idx
        if idx > n_limit:
            return 0
        current_keys = next_keys
        
    return 0

def lcm(numbers: list[int]) -> int:
    lowest_common_multiple = 1
    for i in numbers:
        lowest_common_multiple = lowest_common_multiple * i // gcd(lowest_common_multiple, i)
    return lowest_common_multiple

def solution_part2():
    lines: list[str]
    # lines = get_input_part2()
    # lines = get_input_small()
    # lines = get_input_medium()
    lines = get_input()
    doc: Document = Document.create_from_lines(lines)
    print(f'{len(doc.instructions)} instructions')
    print(f'start_keys: {doc.start_keys}')
    print(f'end_keys: {doc.end_keys}')

    max_iter: int = 100000
    n_cycles: list[int] = []
    current_keys: list[int] = doc.start_keys
    next_keys: list[str]
    for start_key in current_keys:
        print(f'start_key: {start_key}')
        n_last_idx: int = 0
        end_idx_list = []
        current_key = start_key
        for idx, instruction in enumerate(cycle(doc.instructions), start=1):
            next_key = doc.get_key(current_key, instruction)
            if next_key in doc.end_keys:
                print(f'{idx} found end key {next_key}, delta: {idx - n_last_idx}')
                n_last_idx = idx
                end_idx_list.append(idx)
                n_cycles.append(idx)
                break
            current_key = next_key
    print(n_cycles)
    return lcm(n_cycles)
    

solution_part2()